In [1]:
import nest_asyncio

nest_asyncio.apply()

In [3]:
from langchain_community.document_loaders import (
    UnstructuredMarkdownLoader,
)
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import BaseNode, Document, IndexNode, NodeWithScore

from readai.components.retrievers import BM25Retriever


### 数据预处理

#### 加载md文件

In [4]:
# 加载.env文件
import os
from pathlib import Path

from dotenv import load_dotenv

load_dotenv()

# 获取项目根目录路径
project_root = Path(os.getenv("PROJECT_ROOT"))

In [5]:
test_data_path = project_root / "readai/tests/data"

In [6]:
book_name = "comunication_cleaned.md"
book_path = test_data_path / book_name
# 使用unstructured的markdownloader加载
loader = UnstructuredMarkdownLoader(book_path, mode="elements")
documents = loader.load()
# 查看节点数量
print(f"节点数量: {len(documents)}")
# 遍历documents,重构metadata，只保留category，filename这两个属性
for doc in documents:
    category = doc.metadata["category"]
    filename = doc.metadata["filename"]
    doc.metadata = {
        "category": category,
        "filename": filename,
    }

节点数量: 41


In [7]:
llamindex_docs = [
    Document(
        text=doc.page_content,
        metadata=doc.metadata,
        metadata_seperator="::",
        metadata_template="{key}=>{value}",
        text_template="Metadata: {metadata_str}\n-----\nContent: {content}",
    )
    for doc in documents
]

In [8]:
print(len(llamindex_docs))
print(llamindex_docs[0].text, "\n\n", llamindex_docs[0].metadata)

41
马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。 图书在版编目(CIP)数据 非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5 书名原文：Nonviolent Communication ISBN 978-7-5222-0051-4 Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49 中国版本图书馆CIP数据核字（2021）第006542号 Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press. All rights reserved. Used with permission. For further information about Nonviolent Communication(TM) please visit the Center for Nonviolent Communication on the Web at:www.cnvc.org. 版权所有翻印必究 北京市版权局著作权合同登记号：图字01-2016-2253号 非暴力

In [9]:
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.schema import IndexNode

node_parser = SimpleNodeParser.from_defaults(
    chunk_size=1024,
    chunk_overlap=50,
    separator="\n\n",
    secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
)
hybrid_nodes = node_parser.get_nodes_from_documents(llamindex_docs, show_progress=True)

Parsing nodes:   0%|          | 0/41 [00:00<?, ?it/s]

In [10]:
print(hybrid_nodes[0].text)
print(len(hybrid_nodes))

马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。 图书在版编目(CIP)数据 非暴力沟通／（美）马歇尔·卢森堡（Marshall B.Rosenberg）著；刘轶译.-2版（修订本）-北京：华夏出版社有限公司，2021.5 书名原文：Nonviolent Communication ISBN 978-7-5222-0051-4 Ⅰ.①非⋯Ⅱ.①马⋯②刘⋯Ⅲ.①心理交往-通俗读物Ⅳ.①C912.11-49 中国版本图书馆CIP数据核字（2021）第006542号 Translated from the book Nonviolent Communication:A Language of Life 3rd Edition,ISBN 13/10:9781892005281/189200528X by Marshall B.Rosenberg. Copyright ? Fall 2015 Puddle Dancer Press,published by Puddle Dancer Press. All rights reserved. Used with permission. For further information about Nonviolent Communication(TM) please visit the Center for Nonviolent Communication on the Web at:www.cnvc.org.
170


#### small2big方法

In [36]:
sub_chunk_sizes = [128, 512, 2048]
sub_node_parsers = [
    SimpleNodeParser.from_defaults(
        chunk_size=c,
        chunk_overlap=50,
        separator="\n\n",
        secondary_chunking_regex="[^,.;。？！]+[,.;。？！]?",
    )
    for c in sub_chunk_sizes
]

hybrid_all_nodes = []
for hybrid_node in hybrid_nodes:
    for n in sub_node_parsers:
        sub_nodes = n.get_nodes_from_documents([hybrid_node])
        sub_inodes = [
            IndexNode.from_text_node(sn, hybrid_node.node_id) for sn in sub_nodes
        ]
        hybrid_all_nodes.extend(sub_inodes)

    # also add original node to node
    original_node = IndexNode.from_text_node(hybrid_node, hybrid_node.node_id)
    hybrid_all_nodes.append(original_node)

print(len(hybrid_all_nodes))

2753


In [46]:
for node in hybrid_all_nodes[:10]:
    print(node.node_id)

c1b78194-cd59-4627-ac9d-35f12ef4917b
185c1eea-28c0-43db-aca8-31f509108582
f2483f5e-59de-4fa5-94c0-204bbcac24a5
308f0a7e-c6c1-4210-a3e5-d98631330171
7f025097-4068-4459-9ce4-b55cae8599fa
1356a303-99b8-4469-9d1c-1306efabb973
d4405e3c-cc1d-45d5-b633-74f6e0d53385
1a6cb66d-cdde-499e-b9f1-4ef4297257f1
5e59270d-7586-4ce4-a713-2d3b0a4cc006
715598b3-0a5f-4137-96cd-2a17ea80551e


In [40]:
print(type(hybrid_all_nodes[0]))

<class 'llama_index.core.schema.IndexNode'>


In [38]:
hybrid_all_nodes_dict = {n.node_id: n for n in hybrid_all_nodes}

In [52]:
# 查看前5个
for node_id, node in list(hybrid_all_nodes_dict.items())[:5]:
    print(node_id, node.text)

c1b78194-cd59-4627-ac9d-35f12ef4917b 马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。
185c1eea-28c0-43db-aca8-31f509108582 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！
f2483f5e-59de-4fa5-94c0-204bbcac24a5 言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。
308f0a7e-c6c1-4210-a3e5-d98631330171 这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦
7f025097-4068-4459-9ce4-b55cae8599fa 伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。


### 向量库加载

In [14]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.deepseek import DeepSeek

api_key = os.getenv("DEEPSEEK_API_KEY")
model_name = os.getenv("DEEPSEEK_MODEL")
embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5", base_url="http://localhost:11434"
)

# 设置 LLM
llm = DeepSeek(model=model_name, api_key=api_key)

In [17]:
from llama_index.core.indices.vector_store.base import VectorStoreIndex
from llama_index.core.storage import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import AsyncQdrantClient, QdrantClient

client = QdrantClient(url="http://localhost:6333")
aclient = AsyncQdrantClient(url="http://localhost:6333")


In [ ]:
# client.delete_collection("hybrid_nodes")

True

In [18]:
collection_name = "hybrid_nodes"

In [19]:
if client.collection_exists(collection_name):
    print(1)
else:
    print(2)


1


In [42]:
# 如果是传入了nodes类型，直接用类初始化，如果是docs则用from_documents初始化

vector_store_base = QdrantVectorStore(
    client=client,
    aclient=aclient,
    collection_name=collection_name,
)

hybrid_index = None
if client.collection_exists(collection_name):
    print(f"集合 '{collection_name}' 已存在")
    hybrid_index = VectorStoreIndex.from_vector_store(
        vector_store_base, embed_model=embed_model
    )
    print("加载本地向量完成")
else:
    print(f"集合 '{collection_name}' 不存在，需要初始化")
    qdrant_storage_context = StorageContext.from_defaults(
        vector_store=vector_store_base
    )
    hybrid_index = VectorStoreIndex(
        hybrid_all_nodes,
        embed_model=embed_model,
        llm=llm,
        storage_context=qdrant_storage_context,
        show_progress=True,
    )
    print("初始化本地向量完成")

集合 'hybrid_nodes' 不存在，需要初始化


Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/705 [00:00<?, ?it/s]

初始化本地向量完成


#### docstore持久化
DocStore与IndexStore的区别
1.  DocStore (文档存储)
- 存储原始文档节点的**实际内容**
- 包含完整的文本、元数据和节点ID
- 类似于"数据库"，保存所有节点的原始信息

2. IndexStore (索引存储)
- 存储索引的**结构信息**
- 保存索引类型、索引配置和索引之间的关系
- 维护索引的元数据，如索引名称、索引类型
- 包含指向docstore中文档的引用，但不直接存储文档内容

关系说明
两者关系类似于：
- **DocStore**: 图书馆中的所有书籍(实际内容)
- **IndexStore**: 图书馆的分类目录(如何组织和查找这些书籍)

所以当你需要：
- 保留节点唯一ID和内容 → 需要DocStore
- 保留索引结构和检索能力 → 需要IndexStore
- 完整恢复整个系统 → 需要全部存储组件

重要细节
- 从代码中可以看出，向量存储会按命名空间分开存储，每个都有单独的文件
- 持久化会保留完整的对象关系，包括各种引用和指针
- 即使不使用某些存储(如graph_store)，也会创建并持久化一个空的存储对象


In [83]:
from llama_index.core.storage.docstore import SimpleDocumentStore
import os

# 确保docs_persist_dir是目录而非文件
docs_persist_dir = test_data_path / "hybrid_nodes_docstore"

# 持久化时
docstore = SimpleDocumentStore()
docstore.add_documents(hybrid_all_nodes)
docstore.persist(docs_persist_dir)  # 这会自动创建docstore.json文件


In [85]:
# 加载docstore
loaded_docstore = SimpleDocumentStore.from_persist_path(docs_persist_dir)
doc_storage_context = StorageContext.from_defaults(docstore=loaded_docstore)

loaded_docs = doc_storage_context.docstore.docs
for node_id, node in list(loaded_docs.items())[:5]:
    print(node_id, node.text)


c1b78194-cd59-4627-ac9d-35f12ef4917b 马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。
185c1eea-28c0-43db-aca8-31f509108582 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！
f2483f5e-59de-4fa5-94c0-204bbcac24a5 言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。
308f0a7e-c6c1-4210-a3e5-d98631330171 这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦
7f025097-4068-4459-9ce4-b55cae8599fa 伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。


#### VectorStoreIndex持久化（更全的持久处理）

In [43]:
# 持久化
persist_dir = test_data_path / "hybrid_nodes"
# 检查是否存在文件，如果存在说明已经持久化过了，直接加载

hybrid_index.storage_context.persist(persist_dir)
print("持久化完成")


持久化完成


In [50]:
if os.path.exists(persist_dir):
    local_storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
    # 获取node_id等信息时可以用到
    loaded_hybrid_all_nodes_dict = local_storage_context.docstore.docs
    loaded_hybrid_index = local_storage_context.index_store

In [51]:
# 前面的hybrid_all_nodes_dict实际上是一个字典，key是node_id，value是node对象的数据；对应的就是hybrid_all_nodes

for node_id, node in list(loaded_hybrid_all_nodes_dict.items())[:5]:
    print(node_id, node.text)

c1b78194-cd59-4627-ac9d-35f12ef4917b 马歇尔·卢森堡博士发现了一种沟通方式，依照它来谈话和聆听，能使人们情意相通，和谐相处，这就是“非暴力沟通”。 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。
185c1eea-28c0-43db-aca8-31f509108582 做为一个遵纪守法的好人，也许我们从来没有把谈话和“暴力”扯上关系。不过如果稍微留意一下现实生活中的谈话方式，并且用心体会各种谈话方式给我们的不同感受，我们一定会发现，有些话确实伤人！
f2483f5e-59de-4fa5-94c0-204bbcac24a5 言语上的指责、嘲讽、否定、说教以及任意打断、拒不回应、随意出口的评价和结论给我们带来的情感和精神上的创伤甚至比肉体的伤害更加令人痛苦。
308f0a7e-c6c1-4210-a3e5-d98631330171 这些无心或有意的语言暴力让人与人变得冷漠、隔膜、敌视。 非暴力沟通能够： ● 疗愈内心深处的隐秘伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦
7f025097-4068-4459-9ce4-b55cae8599fa 伤痛； ● 超越个人心智和情感的局限性； ● 突破那些引发愤怒、沮丧、焦虑等负面情绪的思维方式； ● 用不带伤害的方式化解人际间的冲突； ● 学会建立和谐的生命体验。


#### BM25检索器准备

- docstore 和 nodes的区别注意区分理解
- 如果用bm25加载从docstore，nodes会产生不一样的结果吗？

In [53]:
import jieba

stop_words_path = project_root / "readai/utils/baidu_stopwords.txt"
jieba.load_userdict(str(stop_words_path))

In [54]:
text = "我来到北京清华大学"
seg_list = jieba.lcut(text, cut_all=False)
print(seg_list)

['我', '来到', '北京', '清华大学']


In [55]:
custom_tokenizer = jieba.Tokenizer()
# 加载自定义词典
custom_tokenizer.load_userdict(str(stop_words_path))
# 使用自定义分词器进行分词
text = "我来到北京清华大学"
seg_list = custom_tokenizer.cut(text, cut_all=False)
print("/ ".join(seg_list))


Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/70/hwlr8qxx6g751fyxb33mcys40000gn/T/jieba.cache
Loading model cost 0.293 seconds.
Prefix dict has been built successfully.


我/ 来到/ 北京/ 清华大学


In [56]:
# 加载stop_words为字符串列表
with open(stop_words_path) as f:
    stop_words = f.readlines()

stop_words = set(stop_words)
print(len(stop_words))

1396


In [57]:
# 如果加载是从docstore会一样吗？
bm25_retriever = BM25Retriever.from_defaults(
    docstore=hybrid_index.docstore,
    similarity_top_k=5,
    tokenizer=custom_tokenizer,
    stopwords=stop_words,
)
# bm25_retriever = BM25Retriever.from_defaults(nodes=hybrid_all_nodes, similarity_top_k=2)


In [58]:
sparse_nodes = bm25_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)
print(len(sparse_nodes))
for n in sparse_nodes:
    display_source_node(n, source_length=1500)

4


**Node ID:** ba4f086f-8c38-43c3-8f3b-9aea68c7ca2e<br>**Similarity:** 26.955977526355234<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** abfe872d-df6f-4439-a582-10fb070a5130<br>**Similarity:** 26.268064015672522<br>**Text:** 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。<br>

**Node ID:** fde69154-05d6-44fb-ba8c-2a93e6c159fe<br>**Similarity:** 26.214797279821816<br>**Text:** 会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。<br>

**Node ID:** 0bdb295b-c32f-4682-9d82-94b7940eb702<br>**Similarity:** 24.495895670924263<br>**Text:** ” 刚开始尝试请他人重述我们的话时，可能会觉得有点不自然。因为在日常交流中，我们很少会提出这样的请求。当我强调请他人重述我们的话是一种重要的能力时，人们通常担心有人会这样反应：“你觉得我是聋子吗？<br>

#### 向量检索器准备

In [59]:
vector_retriever = hybrid_index.as_retriever(similarity_top_k=5)

In [61]:
retrievals = vector_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)
print(len(retrievals))
for n in retrievals:
    display_source_node(n, source_length=1500)

4


**Node ID:** 8e325d94-2b84-41fd-82a3-f6cee61adcf2<br>**Similarity:** 0.6397337<br>**Text:** 我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。<br>

**Node ID:** f6b8857a-e62a-4900-b49f-fc1e872cee87<br>**Similarity:** 0.63871455<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** c2595bd0-7ff4-4fd0-bacc-84de3aa44119<br>**Similarity:** 0.63314337<br>**Text:** 首先，我们观察实际上发生了什么。不论他人的言行是否有益于我们，我们只是去观察。要做到清晰表达所观察到的，我们的挑战在于不夹杂任何评判。不论喜欢与否，我们只是说出人们做了什么。<br>

**Node ID:** 59395a12-aa0e-4224-9011-c3640802d43c<br>**Similarity:** 0.62933075<br>**Text:** 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。<br>

#### small-to-big retriever

In [62]:
from llama_index.core.retrievers import RecursiveRetriever

small2big_retriever = RecursiveRetriever(
    "vector",
    retriever_dict={"vector": vector_retriever},
    node_dict=hybrid_all_nodes_dict,
    verbose=True,
)

In [63]:
# 获取向量召回结果
dense_nodes = small2big_retriever.retrieve(
    "在日常交流中，我们如何区分客观的观察和主观的评价？"
)

Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 83afb5cb-6a2b-4de2-8214-bee270da4552
Retrieving with query id 83afb5cb-6a2b-4de2-8214-bee270da4552: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: b9cabe00-e9b7-4bdb-aedd-9a7992c10a11
Retrieving with query id b9cabe00-e9b7-4bdb-aedd-9a7992c10a11: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: eeac9f1c-d7e5-47eb-b174-ee7be894dd55
Retrieving with query id eeac9f1c-d7e5-47eb-b174-ee7be894dd55: 在日常交流中，我们如何区分客观的观察和主观的评价？


In [64]:
for chunk_node in dense_nodes:
    display_source_node(chunk_node, source_length=1500)


**Node ID:** 83afb5cb-6a2b-4de2-8214-bee270da4552<br>**Similarity:** 0.6397337<br>**Text:** 当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** b9cabe00-e9b7-4bdb-aedd-9a7992c10a11<br>**Similarity:** 0.63314337<br>**Text:** ” “不是这里，”男子回答，“我把钥匙落在巷子里了。” 见到警察很困惑，男子急忙解释：“但这里的光线要亮得多呢。” 我们都对人生拥有渴望。然而我发现，主导我们的文化却可能带我们偏离了方向。为此，我开发了非暴力沟通。它作为一种训练注意力的方式，让我们可以把注意力放在真正要去的地方。我渴望的是，在生命中活出慈悲之心，让由衷的相互给予，在你我之间流动。 那份慈悲心，也是我所说的“由衷的给予”，在我的朋友鲁斯·贝本梅尔（Ruth Bebermeyer）的这段歌词中得到展现： 你取之于我，是对我最好的馈赠。 请你知道，这种给予带给我快乐。 我给予绝非冀盼你偿还， 只因我想活出对你的爱。 也许，你的欣然接受， 就是对我的最佳馈赠。 我无法将两者分开。 你施于我，我给予我的接受； 你取之于我，是对我最好的馈赠。 ——“馈赠”（1978）来自鲁斯·贝本梅尔 唱片集《馈赠》 每当我们想要丰盈他人的生命，由衷给予他人时，我们是出于心中油然而生的喜悦之情。给予者和接受者都会从中获益。当给予是由衷的，而非出于害怕、内疚、羞愧或是渴求回报，接受者才能心无挂碍地享受这份馈赠。与此同时，给予者因为看到自己为他人的幸福做出了贡献，也会收获更多的自我肯定。 我们使用非暴力沟通，并不要求沟通的对方也懂得它，甚至无须对我们保有善意。只要我们遵循非暴力沟通的原则，发自内心地给予，并尽我们所能让对方知道这是我们唯一的动机，他们也会加入我们，最终双方必然能以善意相待。当然，这一切不一定会很快发生，但我相信，只要我们坚持非暴力沟通的原则与方法，友善之花终将在你我心中盛开。 非暴力沟通过程 要达到让双方都能发自内心地相互给予，我们需将注意力聚焦在四个方面——它们是非暴力沟通模式的四个要素。 首先，我们观察实际上发生了什么。不论他人的言行是否有益于我们，我们只是去观察。要做到清晰表达所观察到的，我们的挑战在于不夹杂任何评判。不论喜欢与否，我们只是说出人们做了什么。<br>

**Node ID:** eeac9f1c-d7e5-47eb-b174-ee7be894dd55<br>**Similarity:** 0.62933075<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。语义学家温戴尔·约翰逊（Wendell Johnson）指出，用静态的语言来表达或捕捉瞬息万变的现实世界，会带来许多问题。他说：“我们的语言年代久远，是有缺陷的工具。它反映着泛灵论的思想，倾向于谈论稳定与恒常，谈论相似性、常态和分类，神奇的转变、立竿见影的疗效、简化的问题以及终极解决方案。然而，我们尝试用语言符号化的这个世界，却包含了无穷无尽的过程、变化、维度、变数、关系、功能、互动、发展、学习、应对和复杂性。不断变化的世界与我们相对静态的语言形式并不匹配，我们的许多问题正是因此产生。” 我的同事鲁斯·贝本梅尔在这段歌词中对比了静态语言与动态语言，由此来展现评论与观察的区别： 我从未见过什么懒汉； 我见过的他，有时在白天睡觉， 在某个下雨的日子呆在家里。 但他不是个懒汉。<br>

### 使用HybridRetriever

In [65]:
from llama_index.core.retrievers import QueryFusionRetriever

fusion_retriever = QueryFusionRetriever(
    retrievers=[small2big_retriever, bm25_retriever],
    mode="reciprocal_rerank",
    verbose=True,
    similarity_top_k=10,
    llm=llm,
    num_queries=1,  # 不需要查询扩展
)


In [66]:
nodes = fusion_retriever.retrieve("在日常交流中，我们如何区分客观的观察和主观的评价？")
print(len(nodes))
for node in nodes:
    display_source_node(node, source_length=1500)

2025-04-08 00:24:54.312 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 83afb5cb-6a2b-4de2-8214-bee270da4552
Retrieving with query id 83afb5cb-6a2b-4de2-8214-bee270da4552: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: b9cabe00-e9b7-4bdb-aedd-9a7992c10a11
Retrieving with query id b9cabe00-e9b7-4bdb-aedd-9a7992c10a11: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: eeac9f1c-d7e5-47eb-b174-ee7be894dd55
Retrieving with query id eeac9f1c-d7e5-47eb-b174-ee7be894dd55: 在日常交流中，我们如何区分客观的观察和主观的评价？
7


**Node ID:** 83afb5cb-6a2b-4de2-8214-bee270da4552<br>**Similarity:** 0.016666666666666666<br>**Text:** 当我指出这一点后，另一位老师响应道：“我知道他的意思了。校长的话太多！”这仍不是一个清晰的观察，而是对校长说多少话的评论。随后，第三位老师说：“他认为只有他想说的话是重要的。”我进而向他们解释，推断他人的想法和对他人行为的观察是两码事。随后，第四位老师大胆地说：“他总是想成为人前的焦点。”当我指出这也是推断时，两位老师不约而同地说道：“你的问题太难回答了！” 接着，我们一起拟了份清单，明确列出校长有哪些具体行为令他们感到不满，并确保不掺杂评论。例如，在全体教师会议上，校长会讲述他的童年和战争经历，有时导致会议超时20分钟。我问老师们是否曾经和校长沟通过他们的不满，他们说曾经试过，但都用具有评论意味的言辞向校长提出批评，而从未提及任何具体行为，例如校长在会议中讲述自己的故事。最后，老师们同意在与校长会谈时将这一点提出来。 会谈刚一开始，我便目睹了老师们所描述的情景。不论讨论的主题是什么，校长都会插话说：“想当年……”接着开始讲述他的童年或战争经历。我等着老师们表达他们的不满。然而，他们并没有运用非暴力沟通的方式，而是无声的抗议。有的人开始翻白眼，有的人故意打着哈欠，还有个人一直盯着手表。 直到我按捺不住问他们：“没有人有话要说吗？”迎来的是一阵令人尴尬的沉默。接着，之前会谈中率先发言的那位老师鼓起勇气，直视着校长，然后说出：“艾德，你真是个大嘴巴！” 就像这个故事所展现的，从我们的旧习惯中挣脱出来，并有能力熟练地区分观察与评论并不容易。最终，老师们终于可以明明白白地告诉校长，他们对他的哪些行为感到不满。校长认真地听完后郑重地说：“为什么从没有人告诉我呢？”他承认他有讲故事的习惯，接着就开始说与这个习惯有关的故事了！我见状打断了他的话，婉转地提出他在重蹈覆辙。会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** ba4f086f-8c38-43c3-8f3b-9aea68c7ca2e<br>**Similarity:** 0.016666666666666666<br>**Text:** 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。 续表 注意：总是、永远、从来、每次之类的词语在以下用法中表达的是观察： ·每次我看到杰克打电话，他都至少打半小时。<br>

**Node ID:** b9cabe00-e9b7-4bdb-aedd-9a7992c10a11<br>**Similarity:** 0.01639344262295082<br>**Text:** ” “不是这里，”男子回答，“我把钥匙落在巷子里了。” 见到警察很困惑，男子急忙解释：“但这里的光线要亮得多呢。” 我们都对人生拥有渴望。然而我发现，主导我们的文化却可能带我们偏离了方向。为此，我开发了非暴力沟通。它作为一种训练注意力的方式，让我们可以把注意力放在真正要去的地方。我渴望的是，在生命中活出慈悲之心，让由衷的相互给予，在你我之间流动。 那份慈悲心，也是我所说的“由衷的给予”，在我的朋友鲁斯·贝本梅尔（Ruth Bebermeyer）的这段歌词中得到展现： 你取之于我，是对我最好的馈赠。 请你知道，这种给予带给我快乐。 我给予绝非冀盼你偿还， 只因我想活出对你的爱。 也许，你的欣然接受， 就是对我的最佳馈赠。 我无法将两者分开。 你施于我，我给予我的接受； 你取之于我，是对我最好的馈赠。 ——“馈赠”（1978）来自鲁斯·贝本梅尔 唱片集《馈赠》 每当我们想要丰盈他人的生命，由衷给予他人时，我们是出于心中油然而生的喜悦之情。给予者和接受者都会从中获益。当给予是由衷的，而非出于害怕、内疚、羞愧或是渴求回报，接受者才能心无挂碍地享受这份馈赠。与此同时，给予者因为看到自己为他人的幸福做出了贡献，也会收获更多的自我肯定。 我们使用非暴力沟通，并不要求沟通的对方也懂得它，甚至无须对我们保有善意。只要我们遵循非暴力沟通的原则，发自内心地给予，并尽我们所能让对方知道这是我们唯一的动机，他们也会加入我们，最终双方必然能以善意相待。当然，这一切不一定会很快发生，但我相信，只要我们坚持非暴力沟通的原则与方法，友善之花终将在你我心中盛开。 非暴力沟通过程 要达到让双方都能发自内心地相互给予，我们需将注意力聚焦在四个方面——它们是非暴力沟通模式的四个要素。 首先，我们观察实际上发生了什么。不论他人的言行是否有益于我们，我们只是去观察。要做到清晰表达所观察到的，我们的挑战在于不夹杂任何评判。不论喜欢与否，我们只是说出人们做了什么。<br>

**Node ID:** abfe872d-df6f-4439-a582-10fb070a5130<br>**Similarity:** 0.01639344262295082<br>**Text:** 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。<br>

**Node ID:** eeac9f1c-d7e5-47eb-b174-ee7be894dd55<br>**Similarity:** 0.016129032258064516<br>**Text:** 去观察，就像信仰一样重要。 ——弗雷德里克·布希纳（Frederick Buechner） 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 如果你想把事情搅乱， 我可以告诉你如何做到： 将我做的事情 和你的反应混为一谈。 当你见到做了一半的家务活， 可以告诉我你感到失望。 但说我不负责任， 绝无可能让我做得更多。 当我对你的表白说“不”， 请告诉我你感到伤心。 但说我冷酷无情， 并不能给你带来更多机会。 是的， 我欣然接受你告诉我， 我做了什么或者我未做什么。 我也欣然接受你的评论， 但请不要将两者混淆。 ——马歇尔·卢森堡博士 非暴力沟通的第一个要素是区分观察与评论。我们要清楚地观察有哪些所见、所闻和所触，正影响着我们幸福，而不夹杂任何评论。 在非暴力沟通中，当我们想要清晰且诚恳地向他人表达我们的状态时，“观察”是一个重要的要素。如果我们在观察中夹杂着评论，人们便不那么容易真正听见我们想要表达的内容，反而会听到批评，甚至产生抗拒心理。 非暴力沟通并不要求我们保持完全客观，不做任何评论。它只是强调，我们要区分观察与评论。非暴力沟通是一个动态的语言，它不鼓励一成不变、一概而论的陈述。语义学家温戴尔·约翰逊（Wendell Johnson）指出，用静态的语言来表达或捕捉瞬息万变的现实世界，会带来许多问题。他说：“我们的语言年代久远，是有缺陷的工具。它反映着泛灵论的思想，倾向于谈论稳定与恒常，谈论相似性、常态和分类，神奇的转变、立竿见影的疗效、简化的问题以及终极解决方案。然而，我们尝试用语言符号化的这个世界，却包含了无穷无尽的过程、变化、维度、变数、关系、功能、互动、发展、学习、应对和复杂性。不断变化的世界与我们相对静态的语言形式并不匹配，我们的许多问题正是因此产生。” 我的同事鲁斯·贝本梅尔在这段歌词中对比了静态语言与动态语言，由此来展现评论与观察的区别： 我从未见过什么懒汉； 我见过的他，有时在白天睡觉， 在某个下雨的日子呆在家里。 但他不是个懒汉。<br>

**Node ID:** fde69154-05d6-44fb-ba8c-2a93e6c159fe<br>**Similarity:** 0.016129032258064516<br>**Text:** 会议最后，我们商量了一些办法，以后当老师们不想听校长回忆往事时，就用温和的方式提醒他。 区分观察和评论 在以下列表中，我举例说明如何从混杂着评论的句子中区分出观察。<br>

**Node ID:** 0bdb295b-c32f-4682-9d82-94b7940eb702<br>**Similarity:** 0.015873015873015872<br>**Text:** ” 刚开始尝试请他人重述我们的话时，可能会觉得有点不自然。因为在日常交流中，我们很少会提出这样的请求。当我强调请他人重述我们的话是一种重要的能力时，人们通常担心有人会这样反应：“你觉得我是聋子吗？<br>

In [67]:
from llama_index.core.settings import Settings

Settings.llm = llm

query_engine = RetrieverQueryEngine(fusion_retriever)
response = query_engine.query("在日常交流中，我们如何区分客观的观察和主观的评价？")

2025-04-08 00:25:01.598 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: 83afb5cb-6a2b-4de2-8214-bee270da4552
Retrieving with query id 83afb5cb-6a2b-4de2-8214-bee270da4552: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: b9cabe00-e9b7-4bdb-aedd-9a7992c10a11
Retrieving with query id b9cabe00-e9b7-4bdb-aedd-9a7992c10a11: 在日常交流中，我们如何区分客观的观察和主观的评价？
Retrieved node with id, entering: eeac9f1c-d7e5-47eb-b174-ee7be894dd55
Retrieving with query id eeac9f1c-d7e5-47eb-b174-ee7be894dd55: 在日常交流中，我们如何区分客观的观察和主观的评价？


In [69]:
print(response)

在日常交流中，区分客观观察和主观评价的关键在于：观察是描述具体发生的事实，而评价则包含个人判断或解读。比如"会议超时20分钟"是观察，而"他太啰嗦"就是评价。有效的沟通需要清晰表达观察到的事实，避免将个人判断与事实混为一谈。可以通过具体描述时间、地点、行为等客观要素来保持观察的纯粹性，同时注意避免使用带有价值判断的词汇。这种区分能帮助双方更准确地理解彼此，减少因误解产生的冲突。


## Evaluation

In [70]:
import json
import random

import pandas as pd
from llama_index.core.evaluation import (
    EmbeddingQAFinetuneDataset,
    RetrieverEvaluator,
    generate_question_context_pairs,
)

from readai.components.prompts import QA_GENERATE_PROMPT_TMPL_ZH

In [71]:
QA_GENERATE_PROMPT_TMPL_ZH = """\
##任务：根据以下书籍片段内容，生成具有多维度的讨论性和理解性问题，帮助用户深入理解该段内容。
##Examples:

input_case1:
上下文内容：
---------------------
在《论语》中，孔子提出了“仁”的概念，认为“仁”是人与人之间相互关爱、尊重和理解的基础。他强调，只有具备“仁”的品质，才能真正实现社会的和谐与稳定。
---------------------
请根据上下文生成2个问题
output:
孔子所倡导的“仁”在现代社会中具有怎样的现实意义？
如何理解“仁”是实现社会和谐与稳定的基础？

input_case2:
上下文内容：
---------------------
《物种起源》中，达尔文提出了自然选择学说，认为生物的进化是通过自然选择的过程实现的。他指出，适应环境的个体更容易生存和繁衍后代，从而推动了物种的进化。
---------------------
请根据上下文生成1个问题
output:
适应环境的个体更容易生存和繁衍后代，这一观点如何解释生物多样性的形成？

##下面是用户输入：
上下文内容：
---------------------
{context_str}
---------------------
请根据上下文生成{num_questions_per_chunk}个问题，要求问题至少涉及以下其中一个方面，问题字数长度在100字以内：
1. 片段的核心概念、观点、情节或事件。
2. 作者的意图或观点背后的逻辑推理。
3. 对该段内容的分析、批判或反思。
4. 如果有相关的背景知识或隐含信息，探讨它们的影响。
5. 生成至少一个假设性问题或假设情境，探索不同的可能性。
6. 比较该段内容与其他类似情节或观点的异同。

##格式要求：请参考上面的例子格式生成问题，尽量不携带数字编号并且只需要输出问题，多个问题分行输出即可
##输出：
"""

In [ ]:
print(QA_GENERATE_PROMPT_TMPL_ZH)

In [72]:
print("原节点数", len(hybrid_nodes))

原节点数 170


In [73]:
test_hybrid_nodes = random.sample(hybrid_nodes, 50)
print("测试节点数", len(test_hybrid_nodes))
print(
    f"文本长度：{len(test_hybrid_nodes[0].text)}，节点id：{test_hybrid_nodes[0].node_id}"
)

测试节点数 50
文本长度：851，节点id：e57510a9-e142-4b21-af6a-3eff2905ac1a


In [74]:
eval_dataset_zh = generate_question_context_pairs(
    test_hybrid_nodes, llm=llm, qa_generate_prompt_tmpl=QA_GENERATE_PROMPT_TMPL_ZH
)

100%|██████████| 50/50 [05:11<00:00,  6.24s/it]


In [86]:
test_qa_datasets = test_data_path / "feibaoli_hybrid_eval_dataset50_zh.json"

with open(test_qa_datasets, "w") as f:
    json.dump(eval_dataset_zh.model_dump(), f, indent=4, ensure_ascii=False)

In [87]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]
retriever_evaluator = RetrieverEvaluator.from_metric_names(
    metrics, retriever=fusion_retriever
)

In [90]:
sample_id, sample_query = list(eval_dataset_zh.queries.items())[1]
print(sample_id)
sample_expected = eval_dataset_zh.relevant_docs[sample_id]
print(sample_expected)
eval_result = retriever_evaluator.evaluate(sample_query, sample_expected)
print(eval_result)

2025-04-08 00:46:40.312 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


4ec0062d-03f9-43cc-8f3a-e7ca0294d86e
['e57510a9-e142-4b21-af6a-3eff2905ac1a']
Retrieving with query id None: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: 5ad7a402-1e1b-4000-ad21-e08140a6830b
Retrieving with query id 5ad7a402-1e1b-4000-ad21-e08140a6830b: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: d98ee829-fcea-4589-8dfd-247a00974c25
Retrieving with query id d98ee829-fcea-4589-8dfd-247a00974c25: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: 663f1c65-a3c6-473a-80a8-21dbbfee26fb
Retrieving with query id 663f1c65-a3c6-473a-80a8-21dbbfee26fb: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: 971fe8a3-e056-4ea5-a27e-d8426dd9c0f7
Retrieving with query id 971fe8a3-e056-4ea5-a27e-d8426dd9c0f7: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Query: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Metrics: {'hit_rate': 0.0, 'mrr': 0.

In [91]:
response = query_engine.query(sample_query)
print(response)

2025-04-08 00:46:47.531 | WARNING  | readai.components.retrievers:_retrieve:553 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: 5ad7a402-1e1b-4000-ad21-e08140a6830b
Retrieving with query id 5ad7a402-1e1b-4000-ad21-e08140a6830b: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: d98ee829-fcea-4589-8dfd-247a00974c25
Retrieving with query id d98ee829-fcea-4589-8dfd-247a00974c25: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: 663f1c65-a3c6-473a-80a8-21dbbfee26fb
Retrieving with query id 663f1c65-a3c6-473a-80a8-21dbbfee26fb: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
Retrieved node with id, entering: 971fe8a3-e056-4ea5-a27e-d8426dd9c0f7
Retrieving with query id 971fe8a3-e056-4ea5-a27e-d8426dd9c0f7: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生以下影响：

1. 个人层面：
- 抑郁症状可能减轻，因为个体不再需要通过"辩护式"表达来证明自己需求的合理性
- 减少因长期情感压抑导致的心理健康问题，如案例中女性通过冗长辩解才能提出简单请求的情况
- 增强自我价值感，人们会更相信自己的需求本身就值得被

#### 检测query生成质量

In [92]:
def get_node_by_query_id(dataset: EmbeddingQAFinetuneDataset, query_id):
    """根据query_id查找对应的节点ID和文本内容

    参数:
        dataset: 生成的数据集对象
        query_id: 问题的唯一标识

    返回:
        包含节点ID和文本内容的字典列表
    """
    # 检查query_id是否存在
    if query_id not in dataset.relevant_docs:
        print(f"query_id {query_id} not in dataset.relevant_docs")
        return []

    print(f"query_id {query_id} in dataset.relevant_docs")
    # 输出query_id对应的query
    print(f"对应的query: {dataset.queries[query_id]}")

    # 获取相关节点ID列表
    node_ids = dataset.relevant_docs[query_id]

    # 构建结果
    results = []
    for node_id in node_ids:
        if node_id in dataset.corpus:
            results.append({"node_id": node_id, "text": dataset.corpus[node_id]})

    print("该测试query相关的节点以及内容如下：")
    for node in results:
        print(f"节点ID: {node['node_id']}")
        print(f"节点内容: {node['text']}")
    return results


In [94]:
query_id = "4ec0062d-03f9-43cc-8f3a-e7ca0294d86e"
res_nodes = get_node_by_query_id(eval_dataset_zh, query_id)

query_id 4ec0062d-03f9-43cc-8f3a-e7ca0294d86e in dataset.relevant_docs
对应的query: 如果社会普遍鼓励直接表达需求而非压抑情感，这种文化转变可能对个人抑郁率和社会整体幸福感产生怎样的影响？
该测试query相关的节点以及内容如下：
节点ID: e57510a9-e142-4b21-af6a-3eff2905ac1a
节点内容: 相反，我们被教育成乖孩子、好妈妈、好爸爸。如果我们要成为那样的好人，就得习惯压抑自己。抑郁，是我们做一个“好人”的“代价”。如果你想要让自己好起来，我希望你能清楚地说出，为了让你的生活变得美好，你希望他人做些什么。 来访者：我只是希望有人能爱我。这并非完全不现实，不是吗？ 马歇尔：这是个好的开始。现在我想请你清晰地说出，他人怎么做才能满足你爱的需要？例如，我此刻可以做些什么？ 来访者：哦，你知道的…… 马歇尔：我不确定我知道。我想请你告诉我，为了让你得到你想要的爱，你希望我或者他人怎样做？ 来访者：这很难说清楚。 马歇尔：是的，提出清晰的请求是困难的。但是请想一想，如果我们讲不清楚自己想要什么，别人要回应我们的请求该会有多困难！ 来访者：我开始明白，我希望别人做什么来爱我了。不过，我不太好意思说出来。 马歇尔：是的，这经常是令人尴尬的。那么，你希望我或者他人怎么做？ 来访者：我认真思考了，当我说希望得到爱时，我实际想要的是，即使我还不知道我的需要，你也能猜到我想要什么，并且我希望你一直都这样。 马歇尔：我很感激你做了澄清，我也希望你已经明白了，找到这样一个人用这样的方式来爱你是不可能的。 经过这样的对话，许多来访者才发现，他们之所以感到沮丧和压抑，很大程度上是因为他们不清楚自己希望他人如何来满足自己的需要。 有意识地提出请求 有的时候，表达清晰的请求也许无需过多言语。例如，你正在厨房里，在客厅看电视的妹妹喊道：“我口渴。”在这种情况下，多半她在请求你给她从厨房倒一杯水。 但有时，我们在表达自己的不快时，却误以为对方能明白我们想要什么。例如，一位妻子对丈夫说：“我不是让你在路上带一些黄油和洋葱回来吗？你怎么忘记了？真气人！”也许，妻子的意思是希望丈夫再去一趟商店买东西，但丈夫却很有可能将太太的话视作对他的指责。如果我们只是表达自己的感受，别人可能并不清楚我们想要他们做什

In [316]:
# 对单个节点生成query
num_questions_per_chunk = 5
for node in res_nodes:
    query = QA_GENERATE_PROMPT_TMPL_ZH.format(
        context_str=node["text"], num_questions_per_chunk=num_questions_per_chunk
    )
    print("生成query：", query)
    response = llm.complete(query)
    # result = str(response).strip()
    result = str(response).strip().split("\n")
    print("生成的问题：\n")
    for question in result:
        print(question)

生成query： ##任务：根据以下书籍片段内容，生成具有多维度的讨论性和理解性问题，帮助用户深入理解该段内容。
##Examples:

input_case1:
上下文内容：
---------------------
在《论语》中，孔子提出了“仁”的概念，认为“仁”是人与人之间相互关爱、尊重和理解的基础。他强调，只有具备“仁”的品质，才能真正实现社会的和谐与稳定。
---------------------
请根据上下文生成2个问题
output:
孔子所倡导的“仁”在现代社会中具有怎样的现实意义？
如何理解“仁”是实现社会和谐与稳定的基础？

input_case2:
上下文内容：
---------------------
《物种起源》中，达尔文提出了自然选择学说，认为生物的进化是通过自然选择的过程实现的。他指出，适应环境的个体更容易生存和繁衍后代，从而推动了物种的进化。
---------------------
请根据上下文生成1个问题
output:
适应环境的个体更容易生存和繁衍后代，这一观点如何解释生物多样性的形成？

##下面是用户输入：
上下文内容：
---------------------
我记得最后一天开车回家的路上，我和自己说：“这辈子再也不想身处这样的冲突中了。” 没想到一进家门，我就看到孩子们正在打架。那时，我已经没有力气去同理他们了，便用非暴力沟通的方式大声地呐喊道：“我现在很难受！真的不想来管你们打架的事情，我只想要安静一会儿！”听到这番话，我年仅9岁的大儿子停了下来，看着我问道：“你愿意和我们说说吗？” 我发现，如果我们能坦诚并且不带指责地谈论自己的痛苦，即使对方也处于苦痛之中，有时也能够听见我们的需要。在前述的例子中，我不想冲着孩子大喊：“这怎么回事？你们就不能乖一点吗？我在外面辛苦了一整天才回到家啊！”我也不想用任何其他方式来暗示对方错了。“非暴力呐喊”就是提醒他人注意我在当下的痛苦和迫切的需要。 不过，如果对方也处于强烈的情绪中，无法听见我们，也不肯放过我们，而同时，同理“急救”和“非暴力呐喊”都无法帮助我们的话，我们的另一种选择就是：暂时离开现场，得到自己所需的同理心，等到心境调整好了之后再回去。 小结 所谓“同理”就是带着尊重来理解他人的经历。我们常常有强烈的冲动想给他人建议或安慰，或是解释自己的立场和感受。同理则

### 全测试集评估

In [326]:
eval_results = await retriever_evaluator.aevaluate_dataset(
    eval_dataset_zh, show_progress=True
)

  0%|          | 0/100 [00:00<?, ?it/s]2025-04-07 22:45:37.036 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:37.100 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...


Retrieving with query id None: 在非暴力沟通教学中，作者隐藏不安情绪反而导致学生误解其种族态度，这一案例如何说明"压抑感受"与"误解产生"之间的因果关系？如果作者一开始就坦诚紧张情绪，课堂互动可能会如何不同？
Retrieved node with id, entering: 498020ff-519a-41df-ad64-50f31f731eb4
Retrieving with query id 498020ff-519a-41df-ad64-50f31f731eb4: 在非暴力沟通教学中，作者隐藏不安情绪反而导致学生误解其种族态度，这一案例如何说明"压抑感受"与"误解产生"之间的因果关系？如果作者一开始就坦诚紧张情绪，课堂互动可能会如何不同？
Retrieving with query id None: 如果"小我"的消融真如文中所说是通过深度满足实现的，那么现代社会中追求物质成功的普遍价值观是否与实现"非暴力Ahimsa"存在根本矛盾？这种矛盾该如何调和？
Retrieved node with id, entering: 7d27a3b7-85c9-4458-a3c0-015e1ed33ab6
Retrieving with query id 7d27a3b7-85c9-4458-a3c0-015e1ed33ab6: 如果"小我"的消融真如文中所说是通过深度满足实现的，那么现代社会中追求物质成功的普遍价值观是否与实现"非暴力Ahimsa"存在根本矛盾？这种矛盾该如何调和？
Retrieving with query id None: "不带评论的观察"这一概念在人际沟通或科学研究中如何帮助我们更客观地理解事实？


2025-04-07 22:45:37.160 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
  1%|          | 1/100 [00:00<00:22,  4.39it/s]2025-04-07 22:45:37.207 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:37.259 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:37.312 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...


Retrieved node with id, entering: 4d96d8e2-83d3-4d7e-bbea-d83cfe6736e2
Retrieving with query id 4d96d8e2-83d3-4d7e-bbea-d83cfe6736e2: "不带评论的观察"这一概念在人际沟通或科学研究中如何帮助我们更客观地理解事实？
Retrieved node with id, entering: 92a28caa-a34b-44d1-89d4-67385634d051
Retrieving with query id 92a28caa-a34b-44d1-89d4-67385634d051: "不带评论的观察"这一概念在人际沟通或科学研究中如何帮助我们更客观地理解事实？
Retrieved node with id, entering: 70b4fe18-114b-46da-a5f3-9b099742bcea
Retrieving with query id 70b4fe18-114b-46da-a5f3-9b099742bcea: "不带评论的观察"这一概念在人际沟通或科学研究中如何帮助我们更客观地理解事实？
Retrieving with query id None: 如何区分观察与评论，以及这种区分在有效沟通中的重要性是什么？
Retrieved node with id, entering: 9b0ac4ef-7982-48f3-9295-92499d63a44f
Retrieving with query id 9b0ac4ef-7982-48f3-9295-92499d63a44f: 如何区分观察与评论，以及这种区分在有效沟通中的重要性是什么？
Retrieved node with id, entering: 4d96d8e2-83d3-4d7e-bbea-d83cfe6736e2
Retrieving with query id 4d96d8e2-83d3-4d7e-bbea-d83cfe6736e2: 如何区分观察与评论，以及这种区分在有效沟通中的重要性是什么？
Retrieving with query id None: 如果完全摒弃主观评论进行观察，可能会遇到哪些实践中的挑战或认知局限？
Retrieved node with 

2025-04-07 22:45:37.363 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
  5%|▌         | 5/100 [00:00<00:07, 11.96it/s]2025-04-07 22:45:37.452 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:37.506 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
  7%|▋         | 7/100 [00:00<00:06, 13.71it/s]2025-04-07 22:45:37.553 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...


...

Retrieved node with id, entering: d1919700-ed29-4f1e-87c4-132889fe1b07
Retrieving with query id d1919700-ed29-4f1e-87c4-132889fe1b07: 非暴力沟通强调建立人与人之间的连结是化解冲突的核心，这与传统妥协式解决方式相比有哪些根本性差异？
Retrieved node with id, entering: 88a9d966-607a-4d6d-b257-da16055582fe
Retrieving with query id 88a9d966-607a-4d6d-b257-da16055582fe: 非暴力沟通强调建立人与人之间的连结是化解冲突的核心，这与传统妥协式解决方式相比有哪些根本性差异？
Retrieving with query id None: 在实践非暴力沟通时，如果冲突一方拒绝建立情感连结或坚持"非赢即输"的立场，调解者可以采取哪些策略来突破这种僵局？
Retrieved node with id, entering: d1919700-ed29-4f1e-87c4-132889fe1b07
Retrieving with query id d1919700-ed29-4f1e-87c4-132889fe1b07: 在实践非暴力沟通时，如果冲突一方拒绝建立情感连结或坚持"非赢即输"的立场，调解者可以采取哪些策略来突破这种僵局？
Retrieved node with id, entering: 8dc6ee1f-546f-4be0-b962-94be6d9ecfc0
Retrieving with query id 8dc6ee1f-546f-4be0-b962-94be6d9ecfc0: 在实践非暴力沟通时，如果冲突一方拒绝建立情感连结或坚持"非赢即输"的立场，调解者可以采取哪些策略来突破这种僵局？
Retrieving with query id None: 愤怒的表达方式如何影响人际关系的质量与深度？
Retrieved node with id, entering: 836cd93a-6f22-4ca6-a179-c7e6a9b81e21
Retrieving with query id 836cd93a-6f22-

2025-04-07 22:45:41.897 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:41.945 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:41.999 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:42.049 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
 95%|█████████▌| 95/100 [00:05<00:00, 20.28it/s]

Retrieved node with id, entering: ce429a51-b424-45a4-9c79-23318b7224d1
Retrieving with query id ce429a51-b424-45a4-9c79-23318b7224d1: 作者对"义愤"的批判是否忽略了社会正义运动中集体愤怒的积极作用？
Retrieved node with id, entering: 2a95ea78-a634-4e2f-b3f8-bf0337da949e
Retrieving with query id 2a95ea78-a634-4e2f-b3f8-bf0337da949e: 作者对"义愤"的批判是否忽略了社会正义运动中集体愤怒的积极作用？
Retrieving with query id None: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: fa7e9267-527e-4cbb-89a4-ff9dac43eafc
Retrieving with query id fa7e9267-527e-4cbb-89a4-ff9dac43eafc: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: 4d96d8e2-83d3-4d7e-bbea-d83cfe6736e2
Retrieving with query id 4d96d8e2-83d3-4d7e-bbea-d83cfe6736e2: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: c1da8187-7d49-467d-b04f-9cdf06068727
Retrieving with query id c1da8187-7d49-467d-b04f-9cdf06068727: 如何区分表达真实感受的语言与陈述想法或评论的语言，这种区分在非暴力沟通中的重要性是什么？
Retrieved node with id, entering: 06dceb70-99c6-49ab-

2025-04-07 22:45:42.100 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
2025-04-07 22:45:42.155 | WARNING  | readai.components.retrievers:_retrieve:554 - BM25Retriever does not support embeddings, skipping...
100%|██████████| 100/100 [00:05<00:00, 18.97it/s]

Retrieved node with id, entering: 2032b38a-e43c-41b8-addd-f42b486d6fc6
Retrieving with query id 2032b38a-e43c-41b8-addd-f42b486d6fc6: 如果这对夫妻在冲突中始终无法区分"需要"和"策略"，他们的婚姻关系可能会朝什么方向发展？请结合案例中的具体表现进行分析。
Retrieved node with id, entering: 508d63ce-65d5-4a74-9110-0334fb787bf8
Retrieving with query id 508d63ce-65d5-4a74-9110-0334fb787bf8: 如果这对夫妻在冲突中始终无法区分"需要"和"策略"，他们的婚姻关系可能会朝什么方向发展？请结合案例中的具体表现进行分析。
Retrieving with query id None: "非暴力Ahimsa"作为一种稳定的无我状态，与日常中短暂的无我体验（如欣赏艺术时）本质区别是什么？这种稳定性如何影响个人与社会的关系？
Retrieved node with id, entering: 7d27a3b7-85c9-4458-a3c0-015e1ed33ab6
Retrieving with query id 7d27a3b7-85c9-4458-a3c0-015e1ed33ab6: "非暴力Ahimsa"作为一种稳定的无我状态，与日常中短暂的无我体验（如欣赏艺术时）本质区别是什么？这种稳定性如何影响个人与社会的关系？
Retrieving with query id None: 为什么管理层最初害怕向医生表达脆弱情绪，而实际表达后却取得了意想不到的积极效果？这反映了职场沟通中怎样的心理机制？
Retrieved node with id, entering: 498020ff-519a-41df-ad64-50f31f731eb4
Retrieving with query id 498020ff-519a-41df-ad64-50f31f731eb4: 为什么管理层最初害怕向医生表达脆弱情绪，而实际表达后却取得了意想不到的积极效果？这反映了职场沟通中怎样的心理机制？
Retrieved node with id, en

In [327]:
import pandas as pd


def display_results(name, eval_results):
    """Display results from evaluate."""
    metric_dicts = []
    for eval_result in eval_results:
        metric_dict = eval_result.metric_vals_dict
        metric_dicts.append(metric_dict)

    full_df = pd.DataFrame(metric_dicts)

    columns = {
        "retrievers": [name],
        **{k: [full_df[k].mean()] for k in metrics},
    }

    metric_df = pd.DataFrame(columns)

    return metric_df

In [328]:
display_results("Hybrid_retriever", eval_results)

,retrievers,hit_rate,mrr,precision,recall,ap,ndcg
0,Hybrid_retriever,0.51,0.44,0.102611,0.51,0.44,0.457198


## 自定义实现Retriever

In [129]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever, RecursiveRetriever


class HybridRetriever(BaseRetriever):
    def __init__(
        self,
        dense_retriever: RecursiveRetriever,
        sparse_retriever: BM25Retriever,
        retrieval_type=3,
        topk=256,
    ):
        self.dense_retriever = dense_retriever
        self.sparse_retriever = sparse_retriever
        self.retrieval_type = retrieval_type  # 1:dense only 2:sparse only 3:hybrid
        self.filters = None
        self.filter_dict = None
        self.topk = topk
        super().__init__()

    @classmethod
    def fusion(self, list_of_list_ranks_system, topk=256):
        all_nodes = []

        node_ids = set()
        for nodes in list_of_list_ranks_system:
            for node in nodes:
                content = node.get_content()
                if content not in node_ids:
                    all_nodes.append(node)
                    node_ids.add(content)
        all_nodes = sorted(all_nodes, key=lambda node: node.score, reverse=True)
        topk = min(len(all_nodes), topk)
        # print("simple fusion后数量:", topk)
        return all_nodes[:topk]

    # 倒数排序融合
    @classmethod
    def reciprocal_rank_fusion(self, list_of_list_ranks_system, K=60, topk=256):
        from collections import defaultdict

        rrf_map = defaultdict(float)
        text_to_node = {}
        for rank_list in list_of_list_ranks_system:
            for rank, item in enumerate(rank_list, 1):
                content = item.get_content()
                text_to_node[content] = item
                rrf_map[content] += 1 / (rank + K)
        sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

        reranked_nodes: list[NodeWithScore] = []
        for text, score in sorted_items:
            reranked_nodes.append(text_to_node[text])
            reranked_nodes[-1].score = score
        topk = min(topk, len(reranked_nodes))
        # print("rrf fusion后数量:", topk)
        return reranked_nodes[:topk]

    async def _aretrieve(self, query_bundle: QueryBundle) -> list[NodeWithScore]:
        if self.retrieval_type != 1:
            self.sparse_retriever.filter_dict = self.filter_dict
            sparse_nodes = await self.sparse_retriever.aretrieve(query_bundle)
            if self.retrieval_type == 2:
                return sparse_nodes
        if self.retrieval_type != 2:
            self.dense_retriever.filters = self.filters
            dense_nodes = await self.dense_retriever.aretrieve(query_bundle)
            if self.retrieval_type == 1:
                return dense_nodes

        # combine the two lists of nodes
        # all_nodes = self.fusion(sparse_nodes, dense_nodes)
        all_nodes = self.reciprocal_rank_fusion(
            [sparse_nodes, dense_nodes], topk=self.topk
        )
        return all_nodes

    def _retrieve(self, query_bundle: QueryBundle) -> list[NodeWithScore]:
        sparse_nodes = self.sparse_retriever.retrieve(query_bundle)
        dense_nodes = self.dense_retriever.retrieve(query_bundle)

        # combine the two lists of nodes
        all_nodes = self.reciprocal_rank_fusion(
            [sparse_nodes, dense_nodes], topk=self.topk
        )
        return all_nodes

### Hybrid test

In [ ]:
def reciprocal_rank_fusion(list_of_list_ranks_system, K=60, topk=256):
    from collections import defaultdict

    rrf_map = defaultdict(float)
    text_to_node = {}
    for rank_list in list_of_list_ranks_system:
        for rank, item in enumerate(rank_list, 1):
            content = item.get_content()
            text_to_node[content] = item
            rrf_map[content] += 1 / (rank + K)
    sorted_items = sorted(rrf_map.items(), key=lambda x: x[1], reverse=True)

    reranked_nodes: list[NodeWithScore] = []
    for text, score in sorted_items:
        reranked_nodes.append(text_to_node[text])
        reranked_nodes[-1].score = score
    topk = min(topk, len(reranked_nodes))
    print("rrf fusion后数量:", topk)
    return reranked_nodes[:topk]

In [ ]:
final_results = reciprocal_rank_fusion(
    list_of_list_ranks_system=[sparse_nodes, dense_nodes]
)
for n in final_results:
    print(n.score, "\n", n.text, "\n--------\n")